In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import os

print(os.getpid())
%cd ../

import pylib.mix as mix

18117
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# -------------------------------------------------------------------------------
# --- Create a diagonal matrix with x-coordinates on the main diagonal ---
# -------------------------------------------------------------------------------
mix.reload_module(mix)

t_ = 2.0;  # time interval;
nq_ = 3

N_ = 1 << nq_


# --- form Hamiltonian from Shi-21 paper ---
s2 = N_//2 + 2
s3 = s2 + 1
g = 0.1 + 0.01j
g_cc = np.conjugate(g)
A = np.zeros((N_, N_), dtype=complex)
for ii in range(N_):
    if ii > 0:
        h = ii * (s2 + 1 - ii) * (s3 - s2 + ii)
        h = np.sqrt(h)
        A[ii, ii-1] = - g_cc * h
    if ii < (N_-1):
        h = (ii +1) * (s2 - ii) * (s3 - s2 + ii + 1)
        h = np.sqrt(h)
        A[ii, ii+1] = g * h
A *= 1j

mix.print_matrix(A)

# --- Decompose the diagonal matrix into a tensor product of Z-Pauli gates ---
A_dec_ = mix.hermitian_to_pauli(A, flag_filter = True, small_coef = 1e-14, flag_print_details=True)

--- 0 ---
 0.000+0.000j -0.035+0.346j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j
--- 1 ---
-0.035-0.346j  0.000+0.000j -0.055+0.548j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j
--- 2 ---
 0.000+0.000j -0.055-0.548j  0.000+0.000j -0.069+0.693j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j
--- 3 ---
 0.000+0.000j  0.000+0.000j -0.069-0.693j  0.000+0.000j -0.077+0.775j  0.000+0.000j  0.000+0.000j  0.000+0.000j
--- 4 ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j -0.077-0.775j  0.000+0.000j -0.077+0.775j  0.000+0.000j  0.000+0.000j
--- 5 ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j -0.077-0.775j  0.000+0.000j -0.065+0.648j  0.000+0.000j
--- 6 ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j -0.065-0.648j  0.000+0.000j  0.000+0.000j
--- 7 ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j -0.000+0.000j  0.000+0.000j
------------------------

In [28]:
A_rec = mix.reconstruct_hermitian_from_pauli(A_dec_)
mix.print_matrix(A_rec - A)

--- 0 ---
 0.000+0.000j  0.000-0.000j  0.000+0.000j  0.000+0.000j
--- 1 ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j
--- 2 ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j
--- 3 ---
 0.000+0.000j  0.000+0.000j  0.000+0.000j  0.000+0.000j


In [3]:
# -------------------------------------------------------------
# --- TO RECHECK: Form a circuit to model exp(-1j * t_ * A_dec_) ---
# -------------------------------------------------------------
import pylib.QC as QC
from scipy.linalg import expm
from numpy.linalg import inv

# --- Construction of the circuit ---
qc_dec = QC.Circuit(nq_)
qc_dec.form_from_pauli(A_dec_, t_/10., sel_error_order = 2)

# draw the circuit:
print(qc_dec.form_qiskit_circuit().draw(fold = 160)); 
print("")

# get the matrix representation of the circuit:
U_circ = qc_dec.get_matrix()

# --- reference matrix ---
U_ref = expm(-1.j * t_/10. * A)

# --- Compare matrices ---
mix.print_matrix(U_circ.dot(inv(U_ref)))



# # --- Recheck the construction ---
# max_abs_err = 0.0
# for id_x in range(N_):
#     init_state = np.zeros(N_)
#     init_state[id_x] = 1

#     res_U = Ures.dot(init_state)[0,id_x]
#     res_dir_exp = np.exp(-1.j*t_*x[id_x]) # direct computation of the exponential function;

#     abs_err = np.abs(res_U - res_dir_exp)
#     if abs_err > max_abs_err:
#         max_abs_err = abs_err

# print("max. abs. diff: {:0.3e}".format(max_abs_err))

     ┌───┐                                                                                                                                                  »
q_0: ┤ H ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     ├───┤                                                                                            ┌───┐ ┌───┐                               ┌───┐┌─────┐»
q_1: ┤ H ├───────────────────────────────────────────────────────────────────■─────────────────────■──┤ H ├─┤ H ├────────■───────────────────■──┤ H ├┤ Sdg ├»
     ├───┤┌───────────────┐┌───┐┌─────┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐┌─┴─┐┌───────────────┐┌─┴─┐├───┤┌┴───┴┐┌───┐┌─┴─┐┌─────────────┐┌─┴─┐├───┤└┬───┬┘»
q_2: ┤ H ├┤ Rz(-0.018138) ├┤ H ├┤ Sdg ├┤ H ├┤ Rz(-0.18138) ├┤ H ├┤ S ├┤ H ├┤ X ├┤ Rz(-0.011958) ├┤ X ├┤ H ├┤ Sdg ├┤ H ├┤ X ├┤ Rz(0.11958) ├┤ X ├┤ H ├─┤ S ├─»
     └───┘└───────────────┘└───┘└─────┘└───┘└───────

SystemExit: -1

/media/work/docs/codes/QuCF/scripts-py/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
